In [ ]:
#Imports and dependencies
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4"

from os.path import join, abspath, dirname
import sys
sys.path.insert(0, abspath(join("..", dirname(os.getcwd()))))
         
from collections import defaultdict
import re
import random
import json
import torch
import imageio
from PIL import Image, ImageOps
from tqdm import tqdm_notebook
from skimage import img_as_ubyte
from pytorch3d.structures import Meshes
from pytorch3d.utils import ico_sphere
from pytorch3d.io import load_obj, save_obj
from pytorch3d.datasets import (
    ShapeNetCore,
    collate_batched_meshes
)
from pytorch3d.loss import (
    mesh_laplacian_smoothing, 
    mesh_normal_consistency
)
from pytorch3d.renderer import (
    PerspectiveCameras, RasterizationSettings, MeshRenderer, MeshRasterizer, 
    BlendParams, SoftSilhouetteShader, SoftPhongShader, DirectionalLights, TexturesVertex, 
    TexturesAtlas, HardPhongShader, HardFlatShader, look_at_view_transform, PointLights
)
from pytorch3d.io import load_objs_as_meshes

from dataclasses import dataclass, field, asdict, astuple

import numpy as np
#Plotting Libs
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib as mpl


from synth_dataset.trajectory import cam_trajectory
from synth_dataset.mesh import (
    load_meshes, mesh_random_translation, rotate_mesh_around_axis, 
    translate_mesh_on_axis, scale_mesh
)
from synth_dataset.event_renderer import generate_event_frames

from utils.visualization import plot_trajectory_cameras
from utils.manager import RenderManager, ImageManager


In [ ]:
mpl.rcParams['savefig.dpi'] = 150
mpl.rcParams['figure.dpi'] = 150
#Set the device
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(device)

In [ ]:
managers = [46, 8]
trajectory_path = "data/renders/{0}_{1}_shapenet".format("test", "car")
managers = [RenderManager.from_directory(m, trajectory_path) for m in managers]

In [ ]:
mesh_names = ["car-46", "car-8"]
path = "../data/video-mesh-results"
for m in mesh_names:
    dir_path = os.listdir(join(path, m))[0]
    m_path = join(path, m, dir_path)
    mesh_stuff = torch.load(m_path)
    verts = mesh_stuff['vertices'].unsqueeze(0)
    faces = mesh_stuff['faces'].unsqueeze(0)
    mesh = Meshes(verts=verts, faces=faces)

In [ ]:
cameras = PerspectiveCameras(device=device)

# We will also create a phong renderer. This is simpler and only needs to render one face per pixel.
raster_settings = RasterizationSettings(
    image_size=280, 
    blur_radius=0, 
    faces_per_pixel=100,
    max_faces_per_bin=500000
)
# We can add a point light in front of the object. 
#lights = PointLights(device=device, location=((2., 2.0, 2.0),))
lights = PointLights(
    device=device, 
    location=[[-3, 4, -3], ], 
    diffuse_color=((.4, .4, .4),),
    specular_color=((.4, .4, .4),),
)

flat_renderer = MeshRenderer(
    rasterizer=MeshRasterizer(
        cameras=cameras, 
        raster_settings=raster_settings
    ),
    shader=HardFlatShader(device=device, lights=lights, cameras=cameras)
)

In [ ]:
cars = ['19']
manager_path = "data/renders/{0}_{1}_shapenet".format("test", "chair")
#pmo_path = "../../../photometric-mesh-optim/optimized_mesh/0/"
pmo_path = "../data/video-mesh-results"
for c in cars: 
    #Collect the manager
    manager = RenderManager.from_directory(int(c), manager_path)
    R, T = manager._trajectory
    R, T = R.to(device), T.to(device)
    #Collect the mesh
    dir_path = os.listdir(join(pmo_path, f"chair_{c}"))[0]
    m_path = join(pmo_path, f"chair_{c}", dir_path)
    mesh_stuff = torch.load(m_path)
    verts = mesh_stuff['vertices']
    faces = mesh_stuff['faces']
    center = verts.mean(0)
    verts = verts - center
    scale = max(verts.abs().max(0)[0])
    verts = verts / scale
    verts_rgb = torch.ones_like(verts)
    textures = TexturesVertex(verts_features=[verts_rgb.to(device)])
    mesh = Meshes(verts=[verts], faces=[faces], textures=textures)
    mesh = mesh.to(device)
    mesh = rotate_mesh_around_axis(mesh, [90,90,180], flat_renderer,dist=2, device=device)
    #break
    #Create the cameras for rendering
    images = []
    for i in range(R.shape[0]):
        img_gt = np.array(manager.get_image('phong', i, (224, 224))) / 255
        print(img_gt.shape)
        plt.imshow(img_gt)
        plt.show()
        image = torch.clamp(flat_renderer(mesh, R=R[[i]], T=T[[i]] * 2, device=device), 0, 1)
        image *= 255
        img_np = image.detach().cpu()[...,:3].squeeze().numpy().astype(np.uint8)
        plt.imshow(img_np)
        plt.show()
        img_pil = Image.fromarray(img_np)
        
        save_path = join(pmo_path, f"chair_{c}",'images', f"{i}.png")
        img_pil.save(save_path)

### Re-Rendering obj's

In [ ]:
path_to_obj ="../data/renders/test2_dolphin/001-dolphin_2020-10-29T00:28:24/predicted_meshGT-Pose.obj"
#path_to_obj = "../data/renders/test_chair_shapenet/019-chair_2020-11-08T01:41:02/predicted_meshGT-Pose.obj"
verts, faces, _ = load_obj(path_to_obj)
verts_rgb = torch.ones_like(verts)
textures = TexturesVertex(verts_features=[verts_rgb.to(device)])
mesh = Meshes([verts], [faces.verts_idx], textures)
meshes = mesh.to(device).extend(6)
elev = [30] * 6
azim = torch.linspace(110, 310, 6)
dist = 2
R, T = look_at_view_transform(elev=elev, azim=azim, dist=dist, device=device)
images = torch.clamp(flat_renderer(meshes, R=R, T=T, device=device), 0, 1)
images*=255
images_np = images.detach().cpu()[...,:3].squeeze().numpy().astype(np.uint8)
for count, img in enumerate(images_np):
    if count in [1,2]:
        img_pil = Image.fromarray(img)
        img_pil.save(f'tmp{count}.png')
    plt.imshow(img)
    plt.show()
    
    
    


### Aggregate Results

In [ ]:
import statistics
path_to_test = "../data/renders/test_car_shapenet"
agg_results = {
    'q_mean_error': [],
    't_mean_error': [],
    'q_mean_error_trans': [],
    't_mean_error_trans': [],
    'seg_iou': [],
    'GT-Pose': {
        'mesh_iou': [],
        'chamfer_loss': [],
    },
    'Rot+Trans': {
        'mesh_iou': [],
        'chamfer_loss': [],
    },
    'Trans+LookAt': {
        'mesh_iou': [],
        'chamfer_loss': [],
    }
}
#Collect
for f in os.listdir(path_to_test):
    f_path = join(path_to_test, f)
    path_to_json = join(f_path, 'reconstruction_results.json')
    if not os.path.exists(path_to_json):
        continue
    with open(path_to_json, "r") as f:
        results = json.load(f)
    for k in agg_results.keys():
        if isinstance(agg_results[k], list):
            value = results.get(k, 0)
            if isinstance(value, list): value = value[0]
            agg_results[k].append(value)
        elif isinstance(agg_results[k], dict) and k in results:
            for sub_k in agg_results[k].keys():
                value = results[k].get(sub_k, 0)
                #if value > 30:
                    #print('whoooooo', value)
                    #continue
                agg_results[k][sub_k].append(value)

#print(agg_results['q_mean_error'])  
print(len(agg_results['q_mean_error']))
acc = 0
for idx in range(len(agg_results['q_mean_error'])):
    if agg_results['q_mean_error'][idx] < 45 and agg_results['t_mean_error'][idx] < 1:
        acc += 1
#Aggregate
for k in agg_results.keys():
    if isinstance(agg_results[k], list):
        if k.find('_error') != -1:
        
            agg_results[k] = statistics.median(sorted(agg_results[k])[:110])           
                
        else:
            agg_results[k] = sum(agg_results[k]) / len(agg_results[k])
    elif isinstance(agg_results[k], dict):
        for sub_k in agg_results[k].keys():
            if len(agg_results[k][sub_k]) == 0: continue
            agg_results[k][sub_k] = sum(agg_results[k][sub_k]) / len(agg_results[k][sub_k])
        


In [ ]:
print(acc)

In [ ]:
chair = {'q_mean_error': 54.866579805867055,
 't_mean_error': 0.5388392225075304,
 'q_mean_error_trans': 61.90197501721671,
 't_mean_error_trans': 0.19155412594254098,
 'seg_iou': 0.474987036726448,
 'GT-Pose': {'mesh_iou': 0.3589355796575546,
  'chamfer_loss': 2.796546784894807},
 'Rot+Trans': {'mesh_iou': 0.7112318029006323,
  'chamfer_loss': 25.072881937026978},
 'Trans+LookAt': {'mesh_iou': 0.4695173714842115,
  'chamfer_loss': 3.812564043771653}}

car = {'q_mean_error': 50.346605033409304,
 't_mean_error': 0.5105624865831399,
 'q_mean_error_trans': 56.54089143101711,
 't_mean_error_trans': 0.1783685179927,
 'seg_iou': 0.35914203379212356,
 'GT-Pose': {'mesh_iou': [], 'chamfer_loss': []},
 'Rot+Trans': {'mesh_iou': 0.603664345211453,
  'chamfer_loss': 20.83055844004192},
 'Trans+LookAt': {'mesh_iou': 0.43306054671605426,
  'chamfer_loss': 2.560623920153058}}

car2 = {'q_mean_error': 38.32957077026367,
 't_mean_error': 0.4174906313419342,
 'q_mean_error_trans': 49.968566052849695,
 't_mean_error_trans': 0.4174906313419342,
 'seg_iou': 0.34866725090073375,
 'GT-Pose': {'mesh_iou': 0.27524283102580477,
  'chamfer_loss': 1.6745700046183571},
 'Rot+Trans': {'mesh_iou': [], 'chamfer_loss': []},
 'Trans+LookAt': {'mesh_iou': [], 'chamfer_loss': []}}


### Event Contours

In [ ]:
ev_path = "../data/renders/test_chair_shapenet/019-chair_2020-11-08T01:41:02/events/12_event.png"
ev_path_2 = "../data/renders/test_car_shapenet/017-car_2020-11-08T01:36:43/events/17_event.png"
img_path = "../data/renders/test_chair_shapenet/019-chair_2020-11-08T01:41:02/phong/10_phong.png"
ev = np.array(Image.open(ev_path))
ev2 = np.array(Image.open(ev_path_2))
ev *=4
img = np.array(Image.open(img_path))
image_white = np.all(ev==[0,0,0], axis=-1)
image_other = np.all(ev!=[0,0,0], axis=-1)
ev[image_white] = img[image_white] 
#ev[image_other] *= 2
plt.imshow(ev)
plt.show()
pil_img = Image.fromarray(ev)
pil_img.save('test.png')

In [ ]:
from synth_dataset.trajectory import cam_trajectory
from utils.visualization import plot_trajectory_cameras

pepper = ['elev', 'dist']
variation = ['dist', 'elev', 'azim']
random_start = [ 'azim', 'dist']
R, T = cam_trajectory(variation, pepper, random_start, 22)
cameras = PerspectiveCameras(R=R, T=T, device=device)
#fig = plot_trajectory_cameras(cameras, device)
mesh_p = "../data/meshes/teapot.obj"
verts, faces, _ = load_obj(mesh_p)
verts_rgb = torch.ones_like(verts)
textures = TexturesVertex(verts_features=[verts_rgb.to(device)])
mesh = Meshes([verts], [faces.verts_idx], textures)
mesh = mesh.to(device)
R, T = look_at_view_transform(elev=25, azim=0, dist=1.4, device=device)
image = torch.clamp(flat_renderer(mesh, R=R, T=T , device=device), 0, 1)
image*= 255
image_np = image[...,:3].squeeze().detach().cpu().numpy().astype(np.uint8)
plt.imshow(image_np)
plt.show()
image_pil = Image.fromarray(image_np)
image_pil.save('test.png')

In [ ]:
from pytorch3d.ops import sample_points_from_meshes
from pytorch3d.loss import chamfer_distance

def scale_meshes(pred_meshes, gt_meshes, scale='gt-10'):
    if isinstance(scale, float):
        # Assume scale is a single scalar to use for both preds and GT
        pred_scale = gt_scale = scale
    elif isinstance(scale, tuple):
        # Rescale preds and GT with different scalars
        pred_scale, gt_scale = scale
    elif scale.startswith("gt-"):
        # Rescale both preds and GT so that the largest edge length of each GT
        # mesh is target
        target = float(scale[3:])
        bbox = gt_meshes.get_bounding_boxes()  # (N, 3, 2)
        long_edge = (bbox[:, :, 1] - bbox[:, :, 0]).max(dim=1)[0]  # (N,)
        scale = target / long_edge
        if scale.numel() == 1:
            scale = scale.expand(len(pred_meshes))
        pred_scale, gt_scale = scale, scale
        print(pred_scale, gt_scale)
    else:
        raise ValueError("Invalid scale: %r" % scale)
    pred_meshes = pred_meshes.scale_verts(pred_scale)
    gt_meshes = gt_meshes.scale_verts(gt_scale)
    return pred_meshes, gt_meshes

obj = 'chair'
pmo_path = f"../../../photometric-mesh-optim/optimized_mesh/{obj}"
shapenet_path = f"data/renders/test_{obj}_shapenet"

cl = []

for elem in os.listdir(pmo_path):
    car_num = elem.split('.')[0]
    pmo_path_mesh = join(pmo_path, elem)
    mesh_stuff = torch.load(pmo_path_mesh, map_location=device)
    verts = mesh_stuff['vertices']
    faces = mesh_stuff['faces']
    center = verts.mean(0)
    verts = verts - center
    scale = max(verts.abs().max(0)[0])
    verts = verts / scale
    verts_rgb = torch.ones_like(verts)
    textures = TexturesVertex(verts_features=[verts_rgb.to(device)])
    mesh = Meshes(verts=[verts], faces=[faces], textures=textures)
    mesh_pred = mesh.to(device)
    manager = RenderManager.from_directory(int(car_num), shapenet_path)
    mesh_info = manager.metadata["mesh_info"]
    path = f"../data/ShapeNetCorev2/{mesh_info['synset_id']}/{mesh_info['mesh_id']}/models/model_normalized.obj"
    try:
        verts, faces, aux = load_obj(path, load_textures=True, create_texture_atlas=True)

        mesh_gt = Meshes(
            verts=[verts],
            faces=[faces.verts_idx],
            textures=TexturesAtlas(atlas=[aux.texture_atlas]),
        ).to(device)
    except Exception as e:
        mesh_gt = None
        print('CANNOT COMPUTE CHAMFER LOSS', e)
        continue
    
    mesh_pred_compute, mesh_gt_compute = scale_meshes(mesh_pred.clone(), mesh_gt.clone())
    pcl_pred = sample_points_from_meshes(mesh_pred_compute, num_samples = 5000)
    pcl_gt = sample_points_from_meshes(mesh_gt_compute, num_samples = 5000)
    chamfer_loss = chamfer_distance(pcl_pred, pcl_gt, point_reduction='mean')
    print(chamfer_loss)
    cl.append(chamfer_loss)

In [ ]:
cl = [c[0] for c in cl]
cl_tot = sum(cl) / len(cl)
print(cl_tot)